In [ ]:
import collections
import json
import keras
import numpy as np
import os
import sys
sys.path.append("../ecg")
import scipy.stats as sst

import util
import load

In [ ]:
model_path = "../ecg/saved_model/default/1599908175-269/8.016-0.500-001-0.588-0.760.hdf5"
data_path = "../preprocess_dataset/test_physionet.json"

data = load.load_dataset(data_path)
preproc = util.load(os.path.dirname(model_path))
model = keras.models.load_model(model_path)

In [ ]:
data_path = "../preprocess_dataset/training_physionet.json"
with open("../preprocess_dataset/training_physionet.json", 'r') as fid:
    train_labels = [json.loads(l)['labels'] for l in fid]
counts = collections.Counter(preproc.class_to_int[l[0]] for l in train_labels)
counts = sorted(counts.most_common(), key=lambda x: x[0])
counts = list(zip(*counts))[1]
smooth = 500
counts = np.array(counts)[None, None, :]
total = np.sum(counts) + counts.shape[1]
prior = (counts + smooth) / float(total)

In [ ]:
probs = []
labels = []
for x, y  in zip(*data):
    x, y = preproc.process([x], [y])
    probs.append(model.predict(x))
    labels.append(y)

In [ ]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p / prior, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [ ]:
import sklearn.metrics as skm
report = skm.classification_report(
            ground_truth, preds,
            target_names=preproc.classes,
            digits=3)
scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
print(report)
print("Average {:3f}".format(np.mean(scores[2][:3])))